In [1]:
import pandas as pd

In [2]:
# Read 2023 cvs files competitions
boulder_df = pd.read_csv("./competitions/world_cups_2023/2023_boulder.csv")
lead_df = pd.read_csv("./competitions/world_cups_2023/2023_lead.csv")
speed_df = pd.read_csv("./competitions/world_cups_2023/2023_speed.csv")

### Data Cleaning

In [3]:
boulder_df.head()

,Rank,Name,Unnamed: 2,Country,Qualification,Semi-final,Final,event_name,date,gender
0,1.0,MEJDI,SCHALCK,FRA,4t4z 6 6,1t4z 2 9,2t3z 7 7,IFSC_World_Cup_Hachioji_2023,21 April - 23 April 2023,Men
1,2.0,HANNES,VAN DUYSEN,BEL,3t5z 9 15,1t4z 8 14,1t3z 2 11,IFSC_World_Cup_Hachioji_2023,21 April - 23 April 2023,Men
2,3.0,PAUL,JENFT,FRA,4t5z 10 9,2t4z 11 13,1t3z 3 3,IFSC_World_Cup_Hachioji_2023,21 April - 23 April 2023,Men
3,4.0,KOKORO,FUJII,JPN,4t5z 8 17,1t2z 3 4,0t3z 0 9,IFSC_World_Cup_Hachioji_2023,21 April - 23 April 2023,Men
4,5.0,SORATO,ANRAKU,JPN,3t5z 4 17,1t4z 1 12,0t3z 0 11,IFSC_World_Cup_Hachioji_2023,21 April - 23 April 2023,Men


In [4]:
lead_df.head()

,Rank,Name,Unnamed: 2,Country,Qualification,Semi-final,Final,event_name,date,gender
0,1.0,SASCHA,LEHMANN,SUI,4.03,43+,44+,IFSC_World_Cup_Innsbruck_2023,14 June - 18 June 2023,Men
1,2.0,ALEXANDER,MEGOS,GER,4.03,39+,44+,IFSC_World_Cup_Innsbruck_2023,14 June - 18 June 2023,Men
2,3.0,JAKOB,SCHUBERT,AUT,5.48,43,42+,IFSC_World_Cup_Innsbruck_2023,14 June - 18 June 2023,Men
3,4.0,SORATO,ANRAKU,JPN,2.5,40,42+,IFSC_World_Cup_Innsbruck_2023,14 June - 18 June 2023,Men
4,5.0,MEJDI,SCHALCK,FRA,15.2,40,42+,IFSC_World_Cup_Innsbruck_2023,14 June - 18 June 2023,Men


In [5]:
speed_df.head()

,Rank,Name,Unnamed: 2,Country,Qualification,Final,event_name,date,gender
0,1.0,VEDDRIQ,LEONARDO,INA,4.98 (wr) (asr),5.01,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men
1,2.0,JINBAO,LONG,CHN,5.40,5.12,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men
2,3.0,XINSHANG,WANG,CHN,5.17,5.11,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men
3,4.0,JIANGUO,LONG,CHN,5.130,6.37,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men
4,5.0,EUNCHEOL,SHIN,KOR,5.131,8.69 (5.131),IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men


In [6]:
# Change columns names
boulder_df = boulder_df.rename(columns={"Name": "FName", "Unnamed: 2":"LName"})
lead_df = lead_df.rename(columns={"Name": "FName", "Unnamed: 2":"LName"})
speed_df = speed_df.rename(columns={"Name": "FName", "Unnamed: 2":"LName"})

print(boulder_df.columns)
print(lead_df.columns)
print(speed_df.columns)

Index(['Rank', 'FName', 'LName', 'Country', 'Qualification', 'Semi-final',
       'Final', 'event_name', 'date', 'gender'],
      dtype='object')
Index(['Rank', 'FName', 'LName', 'Country', 'Qualification', 'Semi-final',
       'Final', 'event_name', 'date', 'gender'],
      dtype='object')
Index(['Rank', 'FName', 'LName', 'Country', 'Qualification', 'Final',
       'event_name', 'date', 'gender'],
      dtype='object')


In [10]:
# Change the event name to display only the city and year
boulder_df['event_name'].value_counts()

event_name
IFSC_World_Championships_Bern_2023    242
IFSC_World_Cup_Innsbruck_2023         203
IFSC_World_Cup_Brixen_2023            170
IFSC_World_Cup_Prague_2023            168
IFSC_World_Cup_Hachioji_2023          167
28 April - 30 April 2023              165
IFSC_World_Cup_Salt_Lake_City_2023    126
Name: count, dtype: int64

In [11]:
boulder_df[boulder_df['event_name'] == '28 April - 30 April 2023']

,Rank,FName,LName,Country,Qualification,Semi-final,Final,event_name,date,gender
167,1.0,MEJDI,SCHALCK,FRA,4t5z 9 8,2t3z 2 7,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men,NaN
168,2.0,TOMOA,NARASAKI,JPN,4t5z 8 12,2t3z 6 10,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men,NaN
169,3.0,JONGWON,CHON,KOR,4t4z 7 6,2t2z 2 2,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men,NaN
170,4.0,DOHYUN,LEE,KOR,4t5z 7 11,1t2z 1 9,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men,NaN
171,5.0,MANUEL,CORNU,FRA,4t5z 9 13,1t2z 4 5,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men,NaN
...,...,...,...,...,...,...,...,...,...,...
327,69.0,DASOM,YUN,KOR,0t0z 0 0,NaN,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Women,NaN
328,75.0,PRATEEKSHA,ARUN,IND,0t0z 0 0,NaN,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Women,NaN
329,75.0,WAI-MUI,LING,HKG,0t0z 0 0,NaN,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Women,NaN
330,75.0,ELIZABETH,SEPULVEDA,PUR,0t0z 0 0,NaN,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Women,NaN


In [12]:
# Shifts the values from row 167 to 331 to the right, then copy the values from the semi-final columns to the final column.
starting_row = 167
ending_row = 331

# Shift columns
boulder_df.iloc[starting_row:ending_row+1, 7:] = boulder_df.iloc[starting_row:ending_row+1, 6:-1].values
boulder_df.loc[starting_row:ending_row, 'Final'] = boulder_df.loc[starting_row:ending_row,'Semi-final']

In [13]:
boulder_df.iloc[starting_row:ending_row]

,Rank,FName,LName,Country,Qualification,Semi-final,Final,event_name,date,gender
167,1.0,MEJDI,SCHALCK,FRA,4t5z 9 8,2t3z 2 7,2t3z 2 7,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men
168,2.0,TOMOA,NARASAKI,JPN,4t5z 8 12,2t3z 6 10,2t3z 6 10,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men
169,3.0,JONGWON,CHON,KOR,4t4z 7 6,2t2z 2 2,2t2z 2 2,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men
170,4.0,DOHYUN,LEE,KOR,4t5z 7 11,1t2z 1 9,1t2z 1 9,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men
171,5.0,MANUEL,CORNU,FRA,4t5z 9 13,1t2z 4 5,1t2z 4 5,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Men
...,...,...,...,...,...,...,...,...,...,...
326,69.0,TSENG-SHUN,WONG,HKG,0t1z 0 3,NaN,NaN,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Women
327,69.0,DASOM,YUN,KOR,0t0z 0 0,NaN,NaN,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Women
328,75.0,PRATEEKSHA,ARUN,IND,0t0z 0 0,NaN,NaN,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Women
329,75.0,WAI-MUI,LING,HKG,0t0z 0 0,NaN,NaN,IFSC_World_Cup_Seoul_2023,28 April - 30 April 2023,Women


In [14]:
boulder_df['event_name'].value_counts()

event_name
IFSC_World_Championships_Bern_2023    242
IFSC_World_Cup_Innsbruck_2023         203
IFSC_World_Cup_Brixen_2023            170
IFSC_World_Cup_Prague_2023            168
IFSC_World_Cup_Hachioji_2023          167
IFSC_World_Cup_Seoul_2023             165
IFSC_World_Cup_Salt_Lake_City_2023    126
Name: count, dtype: int64

In [15]:
lead_df['event_name'].value_counts()

event_name
IFSC_World_Championships_Bern_2023    230
IFSC_World_Cup_Innsbruck_2023         200
IFSC_World_Cup_Villars_2023           181
IFSC_World_Cup_Chamonix_2023          176
IFSC_World_Cup_Briançon_2023          143
IFSC_World_Cup_Koper_2023             122
IFSC_World_Cup_Wujiang_2023            93
Name: count, dtype: int64

In [16]:
speed_df['event_name'].value_counts()

event_name
IFSC_World_Championships_Bern_2023    136
IFSC_World_Cup_Villars_2023           133
IFSC_World_Cup_Chamonix_2023          129
IFSC_World_Cup_Seoul_2023             122
IFSC_World_Cup_Jakarta_2023           119
IFSC_World_Cup_Salt_Lake_City_2023    113
IFSC_World_Cup_Wujiang_2023            85
Name: count, dtype: int64

In [17]:
# Normalize event names to city_year
def normalize_event_name(event_name):
   return event_name.split("_", 3)[2:][1]

In [18]:
boulder_df['event_name'] = boulder_df['event_name'].apply(normalize_event_name)
lead_df['event_name'] = lead_df['event_name'].apply(normalize_event_name)
speed_df['event_name'] = speed_df['event_name'].apply(normalize_event_name)

In [19]:
boulder_df['event_name'].value_counts()

event_name
Bern_2023              242
Innsbruck_2023         203
Brixen_2023            170
Prague_2023            168
Hachioji_2023          167
Seoul_2023             165
Salt_Lake_City_2023    126
Name: count, dtype: int64

In [20]:
lead_df['event_name'].value_counts()

event_name
Bern_2023         230
Innsbruck_2023    200
Villars_2023      181
Chamonix_2023     176
Briançon_2023     143
Koper_2023        122
Wujiang_2023       93
Name: count, dtype: int64

In [21]:
speed_df['event_name'].value_counts()

event_name
Bern_2023              136
Villars_2023           133
Chamonix_2023          129
Seoul_2023             122
Jakarta_2023           119
Salt_Lake_City_2023    113
Wujiang_2023            85
Name: count, dtype: int64

In [22]:
# Let's add new columns for start and end date for competitions
def extract_dates(df, date_column):
    # Extract start and end dates
    df[['start_date', 'end_date']] = df[date_column].str.extract(r'(\d+ \w+) - (\d+ \w+ \d+)')

    # Extract the year from the end_date and add it to start_date
    df['start_date'] = df['start_date'] + ' ' + df['end_date'].str.split().str[-1]

    # Convert to datetime data type
    df['start_date'] = pd.to_datetime(df['start_date'], format='%d %B %Y')
    df['end_date'] = pd.to_datetime(df['end_date'], format='%d %B %Y')

    return df

In [23]:
boulder_df = extract_dates(boulder_df, 'date')
lead_df = extract_dates(lead_df, 'date')
speed_df = extract_dates(speed_df, 'date')

In [24]:
# Add a column with the event duration
boulder_df['event_duration'] = (boulder_df['end_date'] - boulder_df['start_date']).dt.days
lead_df['event_duration'] = (lead_df['end_date'] - lead_df['start_date']).dt.days
speed_df['event_duration'] = (speed_df['end_date'] - speed_df['start_date']).dt.days

In [25]:
# Count number of 'dns' occurrences
boulder_df.eq('dns').sum()

Rank              0
FName             0
LName             0
Country           0
Qualification     9
Semi-final        0
Final             0
event_name        0
date              0
gender            0
start_date        0
end_date          0
event_duration    0
dtype: int64

In [26]:
lead_df.eq('dns').sum()

Rank               0
FName              0
LName              0
Country            0
Qualification     12
Semi-final         0
Final              0
event_name         0
date               0
gender             0
start_date         0
end_date           0
event_duration     0
dtype: int64

In [27]:
speed_df.eq('dns').sum()

Rank              0
FName             0
LName             0
Country           0
Qualification     4
Final             0
event_name        0
date              0
gender            0
start_date        0
end_date          0
event_duration    0
dtype: int64

In [28]:
# Store the athtles with 'dns' values for later analysis
boulder_dns = boulder_df[boulder_df['Qualification'] == 'dns']
boulder_dns = boulder_dns.reset_index(drop=True)

lead_dns = lead_df[lead_df['Qualification'] == 'dns']
lead_dns = lead_dns.reset_index(drop=True)

speed_dns = speed_df[speed_df['Qualification'] == 'dns']
speed_dns = speed_dns.reset_index(drop=True)

In [29]:
# Remove athletes with 'dns' status in qualification column for all diciplines
boulder_df = boulder_df[boulder_df['Qualification'] != 'dns']
lead_df = lead_df[lead_df['Qualification'] != 'dns']
speed_df = speed_df[speed_df['Qualification'] != 'dns']

In [30]:
speed_df['qlf1'] = speed_df['Qualification'].str.extract(r'(\d+\.\d+)').astype(float)

In [32]:
speed_df['final1'] = speed_df['Final'].str.extract(r'(\d+\.\d+)').astype(float)

In [33]:
speed_df

,Rank,FName,LName,Country,Qualification,Final,event_name,date,gender,start_date,end_date,event_duration,qlf1,final1
0,1.0,VEDDRIQ,LEONARDO,INA,4.98 (wr) (asr),5.01,Seoul_2023,28 April - 30 April 2023,Men,2023-04-28,2023-04-30,2,4.980,5.01
1,2.0,JINBAO,LONG,CHN,5.40,5.12,Seoul_2023,28 April - 30 April 2023,Men,2023-04-28,2023-04-30,2,5.400,5.12
2,3.0,XINSHANG,WANG,CHN,5.17,5.11,Seoul_2023,28 April - 30 April 2023,Men,2023-04-28,2023-04-30,2,5.170,5.11
3,4.0,JIANGUO,LONG,CHN,5.130,6.37,Seoul_2023,28 April - 30 April 2023,Men,2023-04-28,2023-04-30,2,5.130,6.37
4,5.0,EUNCHEOL,SHIN,KOR,5.131,8.69 (5.131),Seoul_2023,28 April - 30 April 2023,Men,2023-04-28,2023-04-30,2,5.131,8.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,31.0,YENA,PI,KOR,9.73,NaN,Wujiang_2023,22 September - 24 September 2023,Women,2023-09-22,2023-09-24,2,9.730,NaN
833,32.0,KARIN,HAYASHI,JPN,9.88,NaN,Wujiang_2023,22 September - 24 September 2023,Women,2023-09-22,2023-09-24,2,9.880,NaN
834,33.0,SHIVPREET,PANNU,IND,10.17,NaN,Wujiang_2023,22 September - 24 September 2023,Women,2023-09-22,2023-09-24,2,10.170,NaN
835,34.0,ANISHA,VERMA,IND,10.20,NaN,Wujiang_2023,22 September - 24 September 2023,Women,2023-09-22,2023-09-24,2,10.200,NaN
